# Práctica 1

## Preparación del entorno

In [ ]:
import numpy as np
import tensorflow as tf
import keras as ker
import matplotlib.pyplot as plt 
import pandas as pd
import sklearn as sk
import PIL
import os
import random

from PIL import Image
from IPython.display import clear_output # Limpiar output
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder
from numpy import asarray
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.layers.core import Activation
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

### Variables a definir

In [ ]:
# No tocar
num_clases = 10
# La foto es 128x96
xpixel = 128
ypixel = 96
# Tocar
n_neuronas_conv1 = 64
n_neuronas_conv2 = 128
n_neuronas_conv3 = 256
l_rate = 0.15
epoch = 15 
batch = 1

# Preprocesamiento de datos

### Esto coge los datos originales y los divide parece funcionar para model.fit

In [ ]:
# Esto funciona con fit pero necesito investigar

# Mi  teoria de lo que es esto por lo que he leido (OJO NO LO SE REALMENTE) esto crea iteradores que formatean la imagen y la pasan por al model.fit el batch size es en cuanto pedazos pasas
# por cadda iteracion como diferencia y hace el split entre validadcion y training no lose pero salia asi en bastantes sitios 

batch_size = 1 

# this is the augmentation configuration we will use for training     ESTO define como pasas las imagenes puedes modificarlas con parametros yo solo normalizo
train_datagen = ImageDataGenerator(
        rescale=1./255, 
        validation_split=0.2,
        )

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator_t = train_datagen.flow_from_directory(                        # Esto creo que crea un iterador en el directorio que sige las intrucciones mencioanda arriba para pasar los                                                                        # datos a model.fit =/ nose es una paranoia
        directory=r'../Dataset/imgs/train/',  # this is the target directory
        color_mode='rgb',
        target_size=(xpixel, ypixel),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical',
        subset="training")  # since we use binary_crossentropy loss, we need binary labels

train_generator_v = train_datagen.flow_from_directory(
        directory=r'../Dataset/imgs/train/',  # this is the target directory
        color_mode='rgb',
        target_size=(xpixel, ypixel),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical',
        subset="validation") 

### Mismo Concepto pero para test ahora en construccion

In [ ]:
batch_size = 1 
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=r'../Dataset/imgs/test/',

    color_mode='rgb',
    batch_size=batch_size,
    image_size=(xpixel, ypixel),
    shuffle=True,
    seed=123,
)

# RED 1 - No funciona good

In [ ]:
def Modelar_red_1(n_conv1,n_conv2,n_conv3,x,y,lr):
    model = Sequential()

    #### Input Layer ####
    model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same',
                    activation='relu', input_shape=(128, 96, 3)))

    #### Convolutional Layers ####
    model.add(Conv2D(32, (3,3), activation='relu'))
    model.add(MaxPooling2D((2,2)))  # Pooling
    model.add(Dropout(0.2)) # Dropout

    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3,3), activation='relu'))
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3,3), activation='relu'))
    model.add(ker.layers.Activation('relu'))
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(512, (5,5), padding='same', activation='relu'))
    model.add(Conv2D(512, (5,5), activation='relu'))
    model.add(MaxPooling2D((4,4)))
    model.add(Dropout(0.2))

    #### Fully-Connected Layer ####
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10, activation='softmax')) #len(class_subset) en vez de 10, lo he cambiado pero no estoy seguro

    # Compilamos
    adam = ker.optimizers.Adam(learning_rate= lr)
    model.compile(loss=ker.loss.categorical_crossentropy,optimizer=adam,metrics=['accuracy'])

    model.summary() # a handy way to inspect the architecture
    return model

# RED 2 - BUENA DE MOMENTO

In [ ]:
def Modelar_red_2(n_conv1,n_conv2,n_conv3,x,y,lr):
    model=Sequential()
    # Capa input
    model.add(Conv2D(filters= n_conv1, kernel_size=(1,1),input_shape=(x,y,3),padding='same',activation='relu')) 
    # Capas convolucionales
    #   >este bloque se puede seguir añadiendo, quiza con menos neuronas, o menos capas convolucionales, pongo dos por dar un ejemplo nada mas
    # (3,3) es mucho se recomienda unsar 1x1 cuando las img no son mayores de 128x128 la nuestra es 128x96
    model.add(Conv2D(n_conv2,(1,1),activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2))) # pooling
    model.add(Dropout(0.2)) # dropout

    model.add(Conv2D(n_conv3,(1,1),activation='relu',padding='same')) 
    model.add(Conv2D(n_conv3,(1,1),activation='relu',padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2))) 
    model.add(Dropout(0.2)) 

    # Capa fully-connected - MLP
    model.add(Flatten())
    model.add(Dense(512,activation='relu')) # red fully-connected
    model.add(Dropout(0.5))
    model.add(Dense(num_clases, activation='softmax')) # capa de salida(softmaxx) 

    print(model.summary()) # con esto inspeccionamos el modelo, muy comodo

    # Compilamos
    adam = tf.keras.optimizers.Adam(learning_rate= lr) # https://keras.io/api/optimizers/adam/ ?
    model.compile(loss="categorical_crossentropy",optimizer=adam,metrics=['accuracy'])

    return model
model = Modelar_red_2(n_neuronas_conv1,n_neuronas_conv2,n_neuronas_conv3,xpixel,ypixel,l_rate)

# RED 3 - PROBAR

https://github.com/fomorians/distracted-drivers-keras/blob/master/main.py

In [ ]:
def Modelar_red_3(n_conv1,n_conv2,n_conv3,x,y,lr):
    model = Sequential()
    model.add(Conv2D(32, 3, 3, border_mode='same', init='he_normal', input_shape=(3, 640, 480)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(32, 3, 3, border_mode='same', init='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(64, 3, 3, subsample=(2, 2), init='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(64, 3, 3, border_mode='same', init='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(128, 3, 3, subsample=(2, 2), init='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Conv2D(128, 3, 3, border_mode='same', init='he_normal'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(128, activation='sigmoid', init='he_normal'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax', init='he_normal'))
    model.compile(Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

### Y ahora realizamos el fit para entrenar la red

In [ ]:
def Entrenar(m,e,v,epo,b):
 #(model,entrenamiento,validacion, epoch,batch)
        m.fit(
        e,
        validation_data=v,
        epochs= epo,
        batch_size = b, 
        verbose=1 #Esto te imprime un progress bar con informacion 
        ) 
        return m
model = Entrenar(model,train_generator_t,train_generator_v,epoch,batch)
# DATO CURIOSO, .FIT() TIENE UNA OPCION LLAMADA VALIDATION_DATA=, IGUAL NI ES NECESARIO CREAR UN GRUPO DE VALIDACION A PARTE

#### Cargar Modelo

In [ ]:
model = ker.models.load_model('path/to/location') # Carga modelos

#### Guardar Modelo

In [ ]:
model.save('modelo2') # guarda el modelo en la ruta que desees

In [ ]:
model.evaluate(train_generator_v)

#### Evaluar Modelo

In [ ]:
def Evaluar(m,e_test,s_test): # model,entrada_test,salida_test
    return m.evaluate(e_test,s_test,verbose=False)
resultado = Evaluar(model, entrada_test, salida_test) # no existen estas variables todavia

#### Predecir

In [ ]:
prediccion = model.predict(train_generator_test) # no existe esta variable todavia
print(prediccion)